In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# **Image Augmentation**

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img

data_dir = '/content/drive/MyDrive/Faulty_solar_panel'
target_size = (224, 224)

# Count images per class
class_counts = {}
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        num_images = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png'))])
        class_counts[class_name] = num_images

# max_images from the largest class
max_images = max(class_counts.values())
print("Class distribution before augmentation:")
for k, v in class_counts.items():
    print(f"{k}: {v} images")

# Augmentation setup
augmentor = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment images in smaller classes
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png'))]
    current_count = len(images)
    target_count = max_images

    if current_count >= target_count:
        continue  # Skip already-balanced or larger classes

    print(f"\nAugmenting '{class_name}' from {current_count} to {target_count}")

    i = 0
    while current_count + i < target_count:
        img_name = images[i % len(images)]
        img_path = os.path.join(class_path, img_name)

        img = load_img(img_path, target_size=target_size)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        for batch in augmentor.flow(x, batch_size=1):
            new_filename = f"aug_{i}_{img_name}"
            save_img(os.path.join(class_path, new_filename), batch[0])
            break

        i += 1

print("\n All classes are balanced!")


In [ ]:
# After Augmentation counting classes
class_counts = {}

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_path):
        num_images = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
        class_counts[class_folder] = num_images

for class_name, count in class_counts.items():
    print(f"{class_name}: {count} images")

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator # convert float into numarical and normalization

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)

train_images = datagen.flow_from_directory('/content/drive/MyDrive/Faulty_solar_panel',target_size=(224, 224),batch_size=32,class_mode="categorical",subset='training')

test_images = datagen.flow_from_directory('/content/drive/MyDrive/Faulty_solar_panel',target_size=(224, 224),batch_size=32,class_mode="categorical",subset="validation")

In [ ]:
class_name = ["Bird-drop","Clean","Dusty","Electrical-damage","Physical-Damage","Snow-Covered"]

images,labels = next(train_images) #next - iteration

for i in range(16):
  plt.subplot(4,4,i+1)
  plt.imshow(images[i])
  class_labels=class_name[np.argmax(labels[i])]
  plt.title(class_labels)
  plt.grid(True)
  plt.tight_layout()
  plt.axis('off')

plt.show()

# **Custom Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
custom_model = Sequential()

custom_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3))),
custom_model.add(MaxPooling2D(2, 2))

custom_model.add(Conv2D(64, (3, 3), activation='relu'))
custom_model.add(MaxPooling2D(2, 2))

custom_model.add(Conv2D(128, (3, 3), activation='relu'))
custom_model.add(MaxPooling2D(2, 2))

custom_model.add(Flatten())

custom_model.add(Dense(128, activation='relu'))
custom_model.add(Dropout(0.5))
custom_model.add(Dense(100, activation="relu"))

custom_model.add(Dense(6, activation="softmax"))  # 6 classes

In [ ]:
custom_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = custom_model.fit(
    train_images,
    epochs=20,
    validation_data=test_images ,
)

In [ ]:
custom_model.save('solar_model.h5')

In [ ]:
#Prediction using cutom model

from tensorflow.keras.preprocessing import image

img_path = "/content/sample3.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

prediction = custom_model.predict(img_array)
class_names = ["Bird-drop","Clean","Dusty","Electrical-damage","Physical-Damage","Snow-Covered"]
print("Predicted class:", class_names[np.argmax(prediction)])

In [ ]:
# Plotting Accuracy and Loss of the Custom model

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper left")
plt.show()

In [ ]:
y_pred = custom_model.predict(test_images)

In [ ]:
confusion_matrix = tf.math.confusion_matrix(
    labels=test_images.classes,
    predictions=np.argmax(y_pred, axis=1)
)

In [ ]:
# plotting the confusion matrix

sns.heatmap(confusion_matrix,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title("Confusion matrix of Custom model")
plt.show()

# **ReNet18**

In [ ]:
!pip uninstall torch torchvision torchaudio --yes
!pip cache purge
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define transforms (resizing, normalization for ResNet18)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the dataset
full_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Faulty_solar_panel', transform=transform)

# Split the dataset into train and validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f'Training dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import  models


# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters
num_classes = 6  # change to your number of classes
batch_size = 32
learning_rate = 0.001
num_epochs = 10  # you can adjust

#Load ResNet18
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Lists to store accuracy values
train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

# Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images2, labels in train_loader:
        images2 = images2.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        running_loss += loss.item()

    # average training loss and accuracy
    train_accuracy = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

  # Validation Loop
    model.eval()
    correct_val = 0
    total_val = 0
    running_val_loss = 0.0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            # validation loss
            running_val_loss += loss.item()
    # average validation loss and accuracy
    val_accuracy = 100 * correct_val / total
    avg_val_loss = running_val_loss / len(val_loader)

    # Appending the accuracies and losses
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    print(f"Validation Accuracy: {val_accuracy:.2f}%")


In [ ]:
# After training, plot accuracy and loss
plt.figure(figsize=(12, 6))

# Plotting Training and Validation Accuracy of Resnet model
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy', color='blue')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='orange')
plt.title('Training and Validation Accuracy of ResNet18')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

# Plotting Training and Validation Loss of Resnet model
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color='blue')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='orange')
plt.title('Training and Validation Loss ResNet18')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('Resnet_metrics.jpg')
plt.tight_layout()
plt.show()

In [ ]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images2, labels in val_loader:
        images2, labels = images2.to(device), labels.to(device)

        outputs = model(images2)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(all_labels, all_preds)
print(cm)

In [ ]:
# Confusion Matrix of ResNet

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix of ResNet')
plt.savefig('confusion_matrix.jpg')
plt.show()

In [ ]:
# 9. Save the model after training
torch.save(model.state_dict(), '/content/resnet18_2.pth')
print("Model saved successfully!")

# **Prediction Using ResNet Model**

In [ ]:
from torchvision import transforms
from PIL import Image

# Load the trained model
model.eval()

#transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# preprocess image
img_path = '/content/drive/MyDrive/Faulty_solar_panel/Clean/Clean (100).jpg'
image = Image.open(img_path).convert('RGB')
image = transform(image)
image = image.unsqueeze(0)  # Add batch dimension
image = image.to(device)

# Predict
with torch.no_grad():
    output = model(image)
    _, predicted_class = torch.max(output, 1)

# Class names
class_names = ["Bird-drop","Clean","Dusty","Electrical-damage","Physical-Damage","Snow-Covered"]
predicted_label = class_names[predicted_class.item()]

print(f"Predicted Label: {predicted_label}")


In [ ]:
from sklearn.metrics import classification_report

class_names = ["Bird-drop","Clean","Dusty","Electrical-damage","Physical-Damage","Snow-Covered"]
print(classification_report(all_labels, all_preds, target_names=class_names))
